In [5]:
#!/usr/bin/env python
#-*- coding: utf-8- -*-
import warnings 
warnings.filterwarnings("ignore")

#load prerequisite librarys  for running the code 
from osgeo import gdal
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt

import shapely



#get sample data 
dem_data = gdal.Open("ALASKA/Alaska.tif")
#set raster data as array
dem_array = dem_data.GetRasterBand(1).ReadAsArray()
#dem_array[dem_array == 0] = np.nan
#get the geotransform data from the raster
transform = dem_data.GetGeoTransform()
print("finished")
%matplotlib

finished
Using matplotlib backend: TkAgg


In [ ]:
transform

In [2]:
dem_array.shape

(5846, 2917)

In [ ]:
#get raster base data
dem = gdal.Open("ALASKA/Alaska.tif")
print(1)
#set raster data as array
dem_array = dem_data.GetRasterBand(1).ReadAsArray()
print(2)
#get the geotransform data from the raster
transform = dem_data.GetGeoTransform()
print(3)
#get projection data
projection = dem_data.GetProjection()
print(4)
#get the rasters resolution
resolution = transform[1]
print(5)

In [ ]:
transform

TODO:
- read up on threading pools
- write out itterative tpi  hierarchys calculator
- build visualization class
- build class with cleancode
- run tpi for 1000, 300, 50, 10, 30, 500

In [ ]:
class TPI_Land_Form_Classification:
    """
    Class prefroming automated landform recognition and transposition from raster to vector layers
    
    """
    def __init__(self , elevation_model = "" , TPI_Radius_m_lst = []):
        """
        Base init Function
        - TPI_Radius_m_lst: list of tpi values to comput for land feature recognition
        - elevation_model: str the directory for the input elevation_model
        """
        """
        TODO : add all of these natively t
        #!/usr/bin/env python
        #-*- coding: utf-8- -*-
        import warnings 
        warnings.filterwarnings("ignore")

        #load prerequisite librarys  for running the code 
        from osgeo import gdal
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        import shapely"""
        
        self.elevation_model = elevation_model
        self.TPI_Radius_m_lst = TPI_Radius_m_lst
        
        assert type(TPI_Radius_m_lst) == list , "Please enter a list of integer values to TPI_Radius_m_lst"
        assert len(elevation_model) > 0 , "please enter a valid directory string leading to a GeoTiff file"
        
    def get_raster_data(self, elevation_model):
        """
        Function returning numpy array from elevation_model along with resolution of said array
         - elevation_model: str the directory for the input elevation_model
        """
        #get raster base data
        dem = gdal.Open(elevation_model)
        #set raster data as array
        dem_array = dem_data.GetRasterBand(1).ReadAsArray()
        #get the geotransform data from the raster
        transform = dem_data.GetGeoTransform()
        #get projection data
        projection = dem_data.GetProjection()
        #get the rasters resolution
        resolution = transform[1]
        
        return projection ,transform, resolution, dem_array
    
    
    def write_output(self, output_model, output_data, original_transform, original_projection):
        """
        Function writing array to GeoTiff file in user specified directory
        - output_model: str the directory for the output model
        - output_data: numpy array that the user defines
        - original_transform: tupel original raster geotransform
        - original_projection: tupel original raster projection
        """
        # set driver
        driver = gdal.GetDriverByName('GTiff')
        #set output driver and instance new raster
        ds = driver.Create(output_model, output_data.shape[1], output_data.shape[0], 1, gdal.GDT_Float32) 
        #set new raster metadata by old raster metadata
        ds.SetProjection(dem.GetProjection())
        ds.SetGeoTransform(dem.GetGeoTransform())
        ds.GetRasterBand(1).WriteArray(out)
        #remove writen raster from memory
        ds.FlushCache()
        ds = None
        
    #with numpy.mean()
    def downsample_res_avg(self, array, scale):
        """
        Function downsampling elevation model by input scale value, aggrigating values by average
        - array: elevation model as ndarray
        - scale: the value by which the user would like to simplify the input array
        """
        original_width = array.shape[1]
        original_hight = array.shape[0]

        width = int( original_width / scale)
        hight = int(original_hight / scale)

        resized_array = np.zeros(shape = (hight, width, 3) ,dtype = np.uint8)

        for i in range(0, original_hight , scale):
            for j in range(0, original_width , scale):
                resized_array[int(i/scale)-1 , int(j/scale)-1 ] = np.mean(array[i:(i + scale), j:(j + scale)], axis = (0,1))

        return resized_array

    
    def get_hillshade(self, slope_array, aspect_array, azimuth = "90", angle_altitude = 100, ):
        """
        Function hillshade for each cell in input elevation model
        - slope_array: elevation model slope values as ndarray
        - aspect_array:  elevation model aspect values as ndarray
        - azimuth: azimuth direction for hillshade algorithm calculation
        - angle_altitude: altitude for angle calculation in hillshade algorithm
        """
        azimuth = 360 - azimuth
        azimuth_rad = azimuth *np.pi/180
        altitude_rad = angle_altitude*np.pi/180
        shade = np.sin(altitude_rad)*np.sin(slope) + np.cos(altitude_rad)*np.cos(slope)*np.cos((azimuth - np.pi/2)  - aspect)
        hillshade = 255 * (shaded + 1)/2
        return hillshade
        

    
    def get_slope_and_aspect(self,array, in_degrees = True ):
        """
        Function calculating slope and aspect for each cell in input elevation model
        - array: elevation model as ndarray
        - in_degrees: boolean switch for returning slope values ini degrees or decimals
        """
        x,y = np.gradient(array)
        slope = np.pi/2 - np.arctan(np.sqrt(x*x + y*y))
        aspect = np.arctan2(-x,y)
        if in_degrees is True:
            slope = np.degrees(np.arctan(slope))
        return slope, aspect
    
    def view (self, offset_y, offset_x, shape, step = 1):
        print("test 1")
        """
        Function returning two matching numpy views for moving window routines.
        - 'offset_y' and 'offset_x' refer to the shift in relation to the analysed (central) cell 
        - 'shape' are 2 dimensions of the data matrix (not of the window!)
        - 'view_in' is the shifted view and 'view_out' is the position of central cells
        """
        size_y, size_x = shape
        x, y = abs(offset_x), abs(offset_y)

        x_in = slice(x , size_x, step) 
        x_out = slice(0, size_x - x, step)

        y_in = slice(y, size_y, step)
        y_out = slice(0, size_y - y, step)

        # the swapping trick    
        if offset_x < 0: x_in, x_out = x_out, x_in                                 
        if offset_y < 0: y_in, y_out = y_out, y_in

        # return window view (in) and main view (out)
        return np.s_[y_in, x_in], np.s_[y_out, x_out]

        
    def calculate_TPI(self, dem_array, radius_m, resolution):
        """
        Function calculating TPI value of dem array at input radius 
        - dem_array: elevation model as numpy array
        - radius_m: radius in meters for TPI calculation
        - resolution: Resolution of original elevation model
        """
        #radius in pixels
        print("start")
        r = int(np.floor(radius_m/resolution))
        #instance window
        win = np.ones((2* r +1, 2* r +1))
        # win = np.array( [    [0, 1, 1, 1, 0]
        #                      [1, 1, 1, 1, 1],
        #                      [1, 1, 0, 1, 1],
        #                      [1, 1, 1, 1, 1],
        #                      [0, 1, 1, 1, 0]  ])

        # window radius is needed for the function,
        # deduce from window size (can be different for height and width…)
        r_y, r_x  = win.shape[0]//2, win.shape[1]//2
        win[r_y, r_x  ] = 0  # let's remove the central cell 

        #matrices for temporary data
        mx_temp = np.zeros(dem_array.shape)
        mx_count = np.zeros(dem_array.shape)

        # loop through window and accumulate values
        for (y,x), weight in np.ndenumerate(win):

            if weight == 0 : continue  #skip zero values !
            # determine views to extract data 
            view_in, view_out = self.view(y - r_y, x - r_x, dem_array.shape)
            # using window weights (eg. for a Gaussian function)
            mx_temp[view_out] += dem_array[view_in]  * weight

           # track the number of neighbours 
           # (this is used for weighted mean : Σ weights*val / Σ weights)
            mx_count[view_out] += weight

        # this is TPI (spot height – average neighbourhood height)
        out = dem_array - mx_temp / mx_count
        #normalize output
        out = ((((out - out.mean()) / out.std()) * 100) + 0.5)
        return out
    
    
    def set_landform_classes(self, big_TPI_array ,small_TPI_array ,slope_array_deg ):
        """
        Function generating landform class labels as described in http://www.jennessent.com/downloads/TPI-poster-TNC_18x22.pdf 
        - big_TPI_array ,small_TPI_array: larger  and smaller tpi arrays of the same shape 
        - slope_array_deg: array containig slope values in degrees
        """
        #calculate standard deviation 
        std = big_TPI_array.std()
        #instantiate zeros filled array of same size as input to hold tpi calculation valuse
        landform_class_array = np.zeros(big_TPI_array.shape)

        # LF = 5 | Broad Flat Areas | (slope = 0)
        landform_class_array[((small_TPI_array > -std) & (small_TPI_array < std)) & ((big_TPI_array > -std) & (big_TPI_array < std)) & (slope_array_deg <= 5)] = 5

        # LF = 6 | Broad open slopes | (slope > 0)
        landform_class_array[((small_TPI_array > -std) & (small_TPI_array < std)) & ((big_TPI_array > -std) & (big_TPI_array < std)) & (slope_array_deg >= 6)] = 6

        # LF = 7 | flat ridge tops | mesa tops
        landform_class_array[((small_TPI_array > -std) & (small_TPI_array < std)) & (big_TPI_array >= std)] = 7

        # LF = 4 | U-shape valleys
        landform_class_array[((small_TPI_array > -std )& (small_TPI_array < std)) & (big_TPI_array <= -std)] = 4

        # LF = 2 | Lateral midslope | incised drainages
        landform_class_array[(small_TPI_array <= -std) & ((big_TPI_array > -std )& (big_TPI_array < std))] = 2

        # LF = 9 | Lateral midslope |drainage divides
        landform_class_array[(small_TPI_array >= std) & ((big_TPI_array > -std) & (big_TPI_array < std))] = 9

        # LF = 3 | Upland incised |drainages |Stream headwaters
        landform_class_array[ (small_TPI_array <= -std) & (big_TPI_array >= std)] = 3

        # LF = 1 | V-shape river valleys | Deep narrow canyons
        landform_class_array[(small_TPI_array <= -std) & (big_TPI_array <= -std)] = 1

        # LF = 10 | mountain tops | High narrow ridges
        landform_class_array[(small_TPI_array >= std) & (big_TPI_array >= std)] = 10

        # LF = 8 |Local ridge/hilltops within | broad valleys
        landform_class_array[(small_TPI_array >= std) & (big_TPI_array <= -std)] = 8

        return landform_class_array
    
    def visualize(self, array,hillshade):
        """
        Function visualizing the output from the TPI_Land_Form_Classification algorithm
        - array: landfrom class labeled array
        - hillshade: hillshade claculation output ndarray
        """
        %matplotlib
        
        # blend hillshade and landform class arrays
        blend = array*0.5 + hillshade*0.5
        #print array as 
        plt.imshow(array)
        plt.show()
        
        return
    
    def test_fit(self):
        """
        Funcution running a mokup of the TPI_Land_Form_Classification algorithm
        """
        #get dem data
        projection ,transform, resolution, dem_array = self.get_raster_data(elevation_model = self.elevation_model)
        
        #calculate slope and aspect algorithm
        slope_array, aspect_array =  self.get_slope_and_aspect(array = dem_array)
        
        #calculate large_tpi
        tpi_big = self.calculate_TPI(dem_array = dem_array, radius_m = max(self.TPI_Radius_m_lst) ,resolution = resolution)
        tpi_big[np.isnan(tpi_big)] = 0
        
        #calculate smaller_tpi
        ##tpi_small = self.calculate_TPI(dem_array = dem_array, radius_m = max(self.TPI_Radius_m_lst) ,resolution = resolution)
        #tpi_small[np.isnan(tpi_small)] = 0
        
        #calculate landform classes
        #lf_array = self.set_landform_classes(tpi_big , tpi_smalli , slope_array)
        
        return tpi_big


In [ ]:
Test = TPI_Land_Form_Classification(elevation_model = "ALASKA/Alaska.tif", TPI_Radius_m_lst = [10])
lf_array = Test.test_fit()

In [ ]:
plt.imshow(lf_array)
plt.title("lf_array")
plt.colorbar()
plt.show()


In [ ]:
#to do debug:
find out why the function is takign so long and fix

In [ ]:
import numpy as np
m = np.array([True,False ,True])
m2 = np.array([False,False,True])

m & m2
